In [1]:
from neuralnet import *
from checker import *
import copy
import matplotlib.pyplot as plt
import numpy as np

## Data Normalization and Label one hot encoding

In [2]:
# Importing train images 
images, labels = load_data('./', mode = 'train')

# shuffling the training dataset
N = np.random.permutation(np.arange(labels.shape[0]))
images = images[N,:]
labels = labels[N]

# pick the first 10000 data for validation
val_images = images[0:10000,:]
val_labels = labels[0:10000]

# normalizing the images and one_hot encoding the labels (validation set)
val_images = normalize_data(val_images)
val_labels = one_hot_encoding(val_labels)

# leave the rest of the data for training
train_images = images[10000:,:]
train_labels = labels[10000:]

# normalizing the images and one_hot encoding the labels (train set)
train_images = normalize_data(train_images)
train_labels = one_hot_encoding(train_labels)

# Importing test images
test_images, test_labels = load_data('./', mode = 't10k')

# shuffling the test dataset
N = np.random.permutation(np.arange(test_labels.shape[0]))


test_images = test_images[N,:]
test_labels = test_labels[N]

# normalizing the images and one_hot encoding the labels (test set)
test_images = normalize_data(test_images)
test_labels = one_hot_encoding(test_labels)

In [3]:
np.mean(train_images[0])

print(softmax(np.array([1,1,1,4])))
print(np.sum((softmax(np.array([[0.00012121, 0.001131, 0.00002323]])))))

IndexError: tuple index out of range

In [4]:
config = load_config('./')
model = Neuralnetwork(config)

In [5]:
print(model.layers[0].w.shape)
print(len(model.layers))

tests = [(-1, 'b', (0,0)), (-3, 'b', (0,0)), (-1, 'w', (0,0)), (-1, 'w', (1,0)), (0, 'w', (0,0)), (0, 'w', (1,0))]
epsilon = 0.01

for test in tests:
    layer_idx = test[0]
    weight_type = test[1]
    weight_idx = test[2]
    
    seen_classes = set()
    xsamples = []
    ysamples = []
    for i in range(len(train_images)):
        xsample = train_images[i].reshape((-1,1))
        ysample = train_labels[i].reshape((-1,1))
        temp = np.argwhere(ysample)
        yclass = temp[0][0]
        if yclass not in seen_classes:
            ysamples.append(ysample)
            xsamples.append(xsample)
            seen_classes.add(yclass)
        if len(seen_classes) == 10:
            break
    
#     print(ysample.shape)
    
    for i in range(len(ysamples)):
        xsample = xsamples[i]
        ysample = ysamples[i]
        
        print('For sample # ', i + 1)
    
        model1 = copy.deepcopy(model)
        model1.forward(xsample, ysample)
        model1.backward()
        if weight_type == 'w':
            grad1 = model1.layers[layer_idx].v_w[weight_idx]
        elif weight_type == 'b':
            grad1 = model1.layers[layer_idx].v_b[weight_idx]
        
        model0 = copy.deepcopy(model)

        if weight_type == 'w':
            model0.layers[layer_idx].w += epsilon
            output, loss1 = model0.forward(xsample, ysample)
            model0.layers[layer_idx].w -= 2*epsilon
            output, loss2 = model0.forward(xsample, ysample)
            slope = (loss1 - loss2) / (2*epsilon)
        else:
            model0.layers[layer_idx].w += epsilon
            output, loss1 = model0.forward(xsample, ysample)
            model0.layers[layer_idx].w -= 2*epsilon
            output, loss2 = model0.forward(xsample, ysample)
            slope = (loss1 - loss2) / (2*epsilon)

        print('grad1 - slope: ', grad1 - slope)
        print('eps square: ', epsilon**2)
    
#     break

(100, 784)
5
For sample #  1
grad1 - slope:  3.6162186221700837e-09
eps square:  0.0001
For sample #  2
grad1 - slope:  5.94295983861754e-12
eps square:  0.0001
For sample #  3
grad1 - slope:  4.3886699205666715e-12
eps square:  0.0001
For sample #  4
grad1 - slope:  1.0866867281376931e-11
eps square:  0.0001
For sample #  5
grad1 - slope:  8.773060571572904e-10
eps square:  0.0001
For sample #  6
grad1 - slope:  3.001829846499494e-11
eps square:  0.0001
For sample #  7
grad1 - slope:  5.43362630366582e-10
eps square:  0.0001
For sample #  8
grad1 - slope:  2.882462177409585e-11
eps square:  0.0001
For sample #  9
grad1 - slope:  -0.09999999997119201
eps square:  0.0001
For sample #  10
grad1 - slope:  5.249495752721835e-12
eps square:  0.0001
For sample #  1
grad1 - slope:  -13.248887443151725
eps square:  0.0001
For sample #  2
grad1 - slope:  -12.505870057728226
eps square:  0.0001
For sample #  3
grad1 - slope:  2.668221223121525
eps square:  0.0001
For sample #  4
grad1 - slope:  

In [4]:
B = 100
N = train_images.shape[0]
num_epoch = 5

for epoch in range(num_epoch):
    
    indeces = np.random.permutation(np.arange(N))
    
    for i in range(int(N/B)):
        
        xtrain = train_images[indeces[B*i:B*(i+1)],:]
        ytrain = train_labels[indeces[B*i:B*(i+1)],:]
#         print('xtrain shape: ', xtrain.shape)
        #print(model.layers[0].w)
#         print('b shape: ', model.layers[0].b.shape)
        forwarded, loss = model.forward(xtrain.T, ytrain.T)
        print('loss: ', loss)
#         print('forwarded shape: ', forwarded.shape)
#         print(forwarded)
#         print(np.sum(forwarded[:, 1]))
        
        model.backward()
#         break
#     break
        
    


loss:  0.24257664158916808
loss:  0.24060122559616903
loss:  0.2373039923597031
loss:  0.23455283594882423
loss:  0.23079985405713868
loss:  0.23301516713328277
loss:  0.23691691662263079
loss:  0.23814269139301167
loss:  0.2356935415950308
loss:  0.23831761666985068
loss:  0.23299025849120927
loss:  0.232210592629409
loss:  0.23248425223483116
loss:  0.23106073100921473
loss:  0.23272697338855322
loss:  0.23747380934586335
loss:  0.23268782921866532
loss:  0.23485774191764902
loss:  0.24222668650593432
loss:  0.23613573812872185
loss:  0.23094045992702417
loss:  0.2313834856024473
loss:  0.23237647545809256
loss:  0.23843222692123556
loss:  0.2324905445921362
loss:  0.23995592861156284
loss:  0.23503697313803668
loss:  0.2296690031983857
loss:  0.23261072378784894
loss:  0.23305745421696764
loss:  0.23183268464328613
loss:  0.23231515821742726
loss:  0.22704520847289383
loss:  0.236701445573496
loss:  0.2335891320120923
loss:  0.23442019367877162
loss:  0.23039288056684115
loss:  0.23

loss:  0.22808904591732362
loss:  0.22837263448162934
loss:  0.23038323825218363
loss:  0.23077149312437537
loss:  0.23137975614213563
loss:  0.23056016829681003
loss:  0.2384561972338131
loss:  0.23182444901480706
loss:  0.23111061344937817
loss:  0.23558084209060093
loss:  0.22933683871114732
loss:  0.23234853047072918
loss:  0.2298208478383606
loss:  0.22960866061741922
loss:  0.2312483379904047
loss:  0.23033655673149314
loss:  0.23009260863388628
loss:  0.22964194406475893
loss:  0.23035673705267992
loss:  0.23093429266744384
loss:  0.22923246183430337
loss:  0.2307053980288115
loss:  0.231720389607704
loss:  0.22962741488047658
loss:  0.22863543875563655
loss:  0.2284327152181713
loss:  0.22699230692882794
loss:  0.22963974398792902
loss:  0.2324906887487361
loss:  0.23133775744835702
loss:  0.23130552121265388
loss:  0.23285228981483475
loss:  0.23235418175120803
loss:  0.2363489220360648
loss:  0.23464450424363997
loss:  0.2310609199245909
loss:  0.2306455221436828
loss:  0.228

loss:  0.22691502654906248
loss:  0.22884488708270528
loss:  0.22839687160682123
loss:  0.23036033838018438
loss:  0.22898307303667642
loss:  0.22950619530138017
loss:  0.22787927509775402
loss:  0.2295753469935837
loss:  0.2265328689591257
loss:  0.22731800052623466
loss:  0.22699435904339377
loss:  0.2270340334369339
loss:  0.22849581684604717
loss:  0.22595243283122804
loss:  0.2265280326762723
loss:  0.22245959207266736
loss:  0.2288849065137843
loss:  0.22798972241596455
loss:  0.22700115329002826
loss:  0.22477912455275578
loss:  0.2301219831818468
loss:  0.2275125974238165
loss:  0.22565468367050304
loss:  0.2250735681407663
loss:  0.2279026997465344
loss:  0.22740699862725527
loss:  0.22536910258985596
loss:  0.22828980069427446
loss:  0.22709945822422253
loss:  0.22349851662438
loss:  0.22570348050467218
loss:  0.22634943213967346
loss:  0.2245011737487634
loss:  0.22745088794726132
loss:  0.2288474318872641
loss:  0.2254439453825853
loss:  0.22538665006514624
loss:  0.2278815

loss:  0.214573251993007
loss:  0.21111663615615367
loss:  0.2092347671371284
loss:  0.2102265903558251
loss:  0.21094025244759848
loss:  0.21631791226379568
loss:  0.2149004986626273
loss:  0.21597903121374717
loss:  0.20556456552066446
loss:  0.21317443942846023
loss:  0.20932294922495379
loss:  0.20978512963078635
loss:  0.21655871799989385
loss:  0.2120338763444706
loss:  0.2076303194284161
loss:  0.20752704161486915
loss:  0.2073935405671575
loss:  0.20728693477333882
loss:  0.20426758090677946
loss:  0.21187672781790917
loss:  0.21311766547817368
loss:  0.20450486638160048
loss:  0.21719159059973395
loss:  0.2059444001323325
loss:  0.2019361570205457
loss:  0.2108420801836143
loss:  0.2097248357117967
loss:  0.21493466637720907
loss:  0.2032990845811949
loss:  0.2092191669342876
loss:  0.2085879918390961
loss:  0.20753154454173903
loss:  0.20661882002126492
loss:  0.21357708256169133
loss:  0.200513451913403
loss:  0.20332794709091598
loss:  0.2040167322965895
loss:  0.2114407807

loss:  0.18886158765107894
loss:  0.19027463814258716
loss:  0.1937594099125639
loss:  0.19401483605024739
loss:  0.2033614086634982
loss:  0.1948362424185683
loss:  0.19806705148834522
loss:  0.19209279220347034
loss:  0.19267369469820533
loss:  0.19451040396450645
loss:  0.20039908963523784
loss:  0.20289256933016994
loss:  0.19372358087404243
loss:  0.19325425427700907
loss:  0.19328902714275153
loss:  0.19421324931872494
loss:  0.19401986110054315
loss:  0.1967478132156882
loss:  0.19681238538449017
loss:  0.20457778917534744
loss:  0.19772973351120757
loss:  0.19527563568273357
loss:  0.19089314626920187
loss:  0.19730953140854784
loss:  0.19797715569766827
loss:  0.18155273337822
loss:  0.19748927114154669
loss:  0.21083255356035688
loss:  0.19166637714159354
loss:  0.20085236966882644
loss:  0.18851659010939525
loss:  0.19877455988725848
loss:  0.1840419021900316
loss:  0.19692028293296285
loss:  0.18948161706083821
loss:  0.2082920326567794
loss:  0.20656257199845837
loss:  0.1

loss:  0.18796692150756214
loss:  0.18491234573574417
loss:  0.19426592942292356
loss:  0.1890931725796018
loss:  0.18465957353891643
loss:  0.1773560314955312
loss:  0.18283932891246754
loss:  0.19741372044004082
loss:  0.18422515618655882
loss:  0.18537040622172646
loss:  0.1782312582680094
loss:  0.18806655935540903
loss:  0.1828091860652572
loss:  0.19860740691580547
loss:  0.19157634350129985
loss:  0.18056392935461646
loss:  0.1695533658112338
loss:  0.17887458970846185
loss:  0.19965805033601136
loss:  0.18270643576202236
loss:  0.1777861502759383
loss:  0.17404367136499604
loss:  0.19576863860525262
loss:  0.18716158475324837
loss:  0.19163120754605098
loss:  0.1896687932072637
loss:  0.19467450764899427
loss:  0.1961750887431779
loss:  0.1711798554343526
loss:  0.1867693657536204
loss:  0.19289751127495697
loss:  0.18877604181573554
loss:  0.191289746604144
loss:  0.18748594428768353
loss:  0.19290945628061673
loss:  0.1775513419370092
loss:  0.18276159581581275
loss:  0.17500

loss:  0.1850099702244062
loss:  0.193631653568327
loss:  0.16970266632203354
loss:  0.18082217038651915
loss:  0.1683358021960217
loss:  0.19203907684378646
loss:  0.1815160309517116
loss:  0.17293353755174518
loss:  0.18704360208515025
loss:  0.17746960519132765
loss:  0.18435633930545395
loss:  0.18086397242233787
loss:  0.19273402818497085
loss:  0.18757234210205517
loss:  0.18199746244105858
loss:  0.17371622596675057
loss:  0.1688434399544137
loss:  0.19504966324509398
loss:  0.18295342888613478
loss:  0.17693334696100355
loss:  0.17694321194106358
loss:  0.1737805140682811
loss:  0.17589867686616056
loss:  0.1856401688926682
loss:  0.18768531520757706
loss:  0.1856677714707352
loss:  0.17499453835788725
loss:  0.1896277980428427
loss:  0.17820276896919462
loss:  0.1742620917475146
loss:  0.18378856614234956
loss:  0.16537228689401115
loss:  0.17033756828084218
loss:  0.18707403872640221
loss:  0.18007139172525774
loss:  0.1815655560818254
loss:  0.191435400150306
loss:  0.174643

loss:  0.1701588257247946
loss:  0.17707459376031337
loss:  0.17212696666987526
loss:  0.1820257900942131
loss:  0.15955589325020217
loss:  0.1751824259334462
loss:  0.1641250551960948
loss:  0.16546918411779418
loss:  0.19031309467046112
loss:  0.15853129025898352
loss:  0.17592494308942727
loss:  0.17794880762804582
loss:  0.15956097746104814
loss:  0.1957302596086433
loss:  0.173997669666015
loss:  0.17277921010766684
loss:  0.17054038055996726
loss:  0.19278338535592182
loss:  0.16823152608962705
loss:  0.1718230974980903
loss:  0.16336192753047085
loss:  0.1762614823062509
loss:  0.17395142605186345
loss:  0.1940830416883719
loss:  0.16741717848854362
loss:  0.1728598241607932
loss:  0.178060922646793
loss:  0.17058535531857835
loss:  0.17000235815684392
loss:  0.17074427322555627
loss:  0.16390933639901772
loss:  0.18536771216594544
loss:  0.17138912508436152
loss:  0.16898957279837043
loss:  0.1723663080541438
loss:  0.18380785204752237
loss:  0.18390788111919207
loss:  0.187759

In [ ]:
print(model.layers[0].w.shape)

In [ ]:
for i in range(4):
    delta = model.layers[i].backward(5, True, 0.1, 0, 0.01)
    print(delta)

In [ ]:
(model.layers[0]).backwards(5, True, 0.1, 0, 0.01)

In [ ]:
print(np.zeros(10))

In [ ]:
model.layers

In [ ]:
x = np.random.rand(10,4)
y = np.random.rand(10,4)


print((x/y).shape)

In [ ]:
print(np.random.randn(10, 4) * np.sqrt(1 / 4))